In [1]:
import os
os.getcwd()

'C:\\Users\\tkeo1\\OneDrive\\바탕 화면\\coding\\Python'

In [29]:
import pandas as pd
data=pd.read_csv('./data/titanic/train.csv')

In [30]:
print(data)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [31]:
print(data.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64


In [32]:
평균=data['Age'].mean()
print(평균)
data['Age'].fillna(value=30, inplace=True)

29.69911764705882


In [33]:
최빈값=data['Embarked'].mode()
print(최빈값)
data['Embarked'].fillna(value='S',inplace=True)

0    S
Name: Embarked, dtype: object


In [34]:
import tensorflow as tf

정답=data.pop('Survived')


ds=tf.data.Dataset.from_tensor_slices((dict(data),정답))
print(ds)


<TensorSliceDataset element_spec=({'PassengerId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Sex': TensorSpec(shape=(), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Embarked': TensorSpec(shape=(), dtype=tf.string, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [35]:
feature_columns=[]

#숫자로 Fare Parch SibSp : numeric_column
#범위로 Age : bucketized_column
#종류 몇개 없는 카테고리화 Sex Embarked Pclass : indicator_column
#종류가 많은 카테고리 Ticket : embedding_column
def 노말라이저함수(x):
    최소=data['Fare'].min()
    최대=data['Fare'].max()
    return (x-최소)/(최대-최소)
feature_columns.append(tf.feature_column.numeric_column('Fare',normalizer_fn=노말라이저함수))
feature_columns.append(tf.feature_column.numeric_column('Parch'))
feature_columns.append(tf.feature_column.numeric_column('SibSp'))
# feature_columns.append(tf.feature_column.numeric_column('Age'))

Age=tf.feature_column.numeric_column('Age')
Age_bucket=tf.feature_column.bucketized_column(Age,boundaries=[10,20,30,40,50,60])
(tf.feature_column.numeric_column('Age_bucket'))
print(Age)
print(feature_columns)

NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
[NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [36]:
vocab=data['Sex'].unique()
cat=tf.feature_column.sequence_categorical_column_with_vocabulary_list('Sex',vocab)
one_hot=tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)

vocab=data['Embarked'].unique()
cat=tf.feature_column.sequence_categorical_column_with_vocabulary_list('Embarked',vocab)
one_hot=tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)

vocab=data['Pclass'].unique()
cat=tf.feature_column.sequence_categorical_column_with_vocabulary_list('Pclass',vocab)
one_hot=tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)

#embedding _ ticket
vocab=data['Ticket'].unique()
cat=tf.feature_column.sequence_categorical_column_with_vocabulary_list('Ticket',vocab)
one_hot=tf.feature_column.embedding_column(cat,dimension=9)
feature_columns.append(one_hot)


In [39]:
ds_batch=ds.batch(32)
next(iter(ds_batch))[0]
feature_layer=tf.keras.layers.DenseFeatures(tf.feature_column.numeric_column('Fare'))
feature_layer(next(iter(ds_batch))[0])

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[  7.25  ],
       [ 71.2833],
       [  7.925 ],
       [ 53.1   ],
       [  8.05  ],
       [  8.4583],
       [ 51.8625],
       [ 21.075 ],
       [ 11.1333],
       [ 30.0708],
       [ 16.7   ],
       [ 26.55  ],
       [  8.05  ],
       [ 31.275 ],
       [  7.8542],
       [ 16.    ],
       [ 29.125 ],
       [ 13.    ],
       [ 18.    ],
       [  7.225 ],
       [ 26.    ],
       [ 13.    ],
       [  8.0292],
       [ 35.5   ],
       [ 21.075 ],
       [ 31.3875],
       [  7.225 ],
       [263.    ],
       [  7.8792],
       [  7.8958],
       [ 27.7208],
       [146.5208]], dtype=float32)>

In [40]:
model=tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
ds_batch=ds.batch(32)

model.fit(ds,validation_data=(),shuffle=True,epochs=20)


Epoch 1/20


ValueError: in user code:

    File "C:\Users\tkeo1\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\tkeo1\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\tkeo1\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\tkeo1\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\tkeo1\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "dense_features_6" "                 f"(type DenseFeatures).
    
    In ind